In [1]:
# import ample
# import scanorama
# import pkg_resources
# pkg_resources.get_distribution("scanorama").version
# !pip show scanorama
# ??scanorama.correct
#  When `return_dimred=False`, returns a two-tuple containing a list of
#         `np.ndarray` with integrated low-dimensional embeddings and a list
#         of `scanpy.api.AnnData` with batch corrected values in the `.X`
#         field.

In [2]:
# Scanorama batch effect correction
# See more at: https://github.com/brianhie/scanorama
# Hoa Tran 
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import time
from datetime import timedelta
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

scanpy==1.4+18.gaabe446 anndata==0.6.17 numpy==1.15.4 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
# Create folder to save the results 
import os
dirname = os.getcwd()
print(dirname)


if not os.path.exists('./results/'): os.makedirs('./results/')   
if not os.path.exists('./results/result_dataset8_mouse_brain/'): os.makedirs('./results/result_dataset8_mouse_brain/')    
save_dir = os.path.join(dirname, 'results/result_dataset8_mouse_brain/')
print(save_dir)

# Function to save figure as image in the figures folder
save_fig_dir='./figures/dataset8_mouse_brain/'
if not os.path.exists('./figures/'): os.makedirs('./figures/')
if not os.path.exists(save_fig_dir): os.makedirs(save_fig_dir)
def save_images(filename, save_fig_dir):    
    outname = save_fig_dir + filename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python
/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python/results/result_dataset8_mouse_brain/


In [4]:
print('Read data')
data_dir = os.path.join('/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/dataset/dataset8_Mouse_brain/raw_data_python/')
savefn = 'dataset8.h5ad'
adata = sc.read_h5ad(os.path.join(data_dir,savefn))
adata

Read data


AnnData object with n_obs × n_vars = 833206 × 17745 
    obs: 'cell_type', 'batch'

In [ ]:
# ds_dir = '/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/xiaomeng/generate_PCA_tSNE_UMAP/dataset8/'
# myDatafn = 'dataset8_raw_pca.csv'
# myPCADS = pd.read_csv(os.path.join(ds_dir, myDatafn),header=0, index_col=0)
# print(myPCADS.values.shape)
# print(myPCADS.keys())
# print(myPCADS.index[1:5])
# adata.X[1:3,1:3]

In [5]:
adata


AnnData object with n_obs × n_vars = 833206 × 17745 
    obs: 'cell_type', 'batch'

In [9]:
print('Read data')
data_dir = '/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/dataset/dataset8_Mouse_brain/raw_data_python/'
savefnds = 'downsample_d8_10percent.h5ad'
adatads = sc.read_h5ad(os.path.join(data_dir,savefnds))
sc.pp.normalize_per_cell(adatads)
sc.pp.log1p(adatads)
adatads

Read data


AnnData object with n_obs × n_vars = 83323 × 17745 
    obs: 'cell_type', 'batch', 'n_counts'

In [10]:
sc.pp.highly_variable_genes(adatads, n_top_genes=5000, flavor='seurat')
print([sum(adatads.var['highly_variable']),len(adatads.var['highly_variable'])])


--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
[5000, 17745]


In [11]:
# Extract data from batch 1 and batch 2
# Already filtered
# sc.pp.filter_cells(adata, min_genes=300)
# sc.pp.filter_genes(adata, min_cells=10)

sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

adata1 = adata[adata.obs['batch']=='batch1',:].copy()
adata1 = adata1[:,adatads.var['highly_variable']]
adata2 = adata[adata.obs['batch']=='batch2',:].copy()
adata2 = adata2[:,adatads.var['highly_variable']]
print(adata1)
print(adata2)
# sum(adata2.var_names==adata1.var_names)


View of AnnData object with n_obs × n_vars = 691600 × 5000 
    obs: 'cell_type', 'batch', 'n_counts'
View of AnnData object with n_obs × n_vars = 141606 × 5000 
    obs: 'cell_type', 'batch', 'n_counts'


In [17]:
adata2

View of AnnData object with n_obs × n_vars = 141606 × 5000 
    obs: 'cell_type', 'batch', 'n_counts'

In [12]:
from scanorama import correct, visualize, process_data
from scanorama import dimensionality_reduce
from scanorama import *
import scanorama


In [ ]:
# process_data(datasets, genes, hvg=HVG, dimred=DIMRED, verbose=False)
# datasets_dimred, genes = process_data(datasets, genes, verbose=True)

In [13]:
#  corrected, corrected_be = scanorama.correct_scanpy(adata_ls, batch_size=80, 
#                                                       return_dimred=True, geosketch=True, geosketch_max=6900)
# from ample import gs, uniform
# Can not work 
# !pip install scanorama --upgrade

adata_ls = [adata1, adata2]

In [14]:
# integrated, corrected
# When `return_dimred=True`, returns a two-tuple containing a list of
# `np.ndarray` with integrated low-dimensional embeddings and a list
# of new `scanpy.api.AnnData`.

# Batch correction.
# datasets = [df.values]
# List of datasets (matrices of cells-by-genes)
t1 = time.time()
# corrected = scanorama.correct_scanpy(adata_ls, batch_size=60, return_dense=True, knn=15)
# corrected, genes = scanorama.correct(adata_ls, adata1.var_names,batch_size=30)
corrected, corrected_be = scanorama.correct_scanpy(adata_ls, batch_size=80, 
                                                   return_dimred=True,
                                                   geosketch=True, geosketch_max=6900)

# integrated, corrected = scanorama.correct_scanpy(adatas, return_dimred=True)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))

Found 5000 genes among all datasets
[[0.         0.16884058]
 [0.         0.        ]]
Processing datasets (0, 1)
Took 0:11:57.010815


In [18]:
print(corrected_be[0])
print(corrected_be[1])

AnnData object with n_obs × n_vars = 691600 × 5000 
AnnData object with n_obs × n_vars = 141606 × 5000 


In [15]:
def getExecutionTime(t1, t2, save_dir, usecase_name,filename):
    time_taken = (t2 - t1)
    time_taken_mins = divmod(time_taken, 60)
    time_taken_hours, rest = divmod( time_taken, 3600)
    hours_mins, hours_secs = divmod( rest, 60)
    print('Took seconds: '+str(timedelta(seconds=round(time_taken))))
    print('Took minutes: '+str(time_taken_mins))
    print('Took hours_minutes_seconds: ',str(time_taken_hours),str(hours_mins),str(hours_secs))
    
    

    data = {'use_case':usecase_name, 'exetime_secs':str(round(time_taken)),
           'exetimehours': str(time_taken_hours),
           'exetimemins': str(hours_mins),
           'exetimesecs':str(round(hours_secs))} 
    df = pd.DataFrame(data, index =['exetime'])
    print(df)
    df.to_csv(save_dir+filename) 

# Evaluation runtime of main batch effect removal function
filename = 'scanorama_exetime.csv'
usecase_name = 'scanorama_exetime' 
getExecutionTime(t1, t2, save_dir, usecase_name, filename)  # t1: start time, t2: end time  
print(save_dir)

Took seconds: 0:11:57
Took minutes: (11.0, 57.01081466674805)
Took hours_minutes_seconds:  0.0 11.0 57.01081466674805
                  use_case exetime_secs exetimehours exetimemins exetimesecs
exetime  scanorama_exetime          717          0.0        11.0          57
/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python/results/result_dataset8_mouse_brain/


In [ ]:
del adata_ls


In [ ]:
corrected_adata = corrected_be[0].concatenate(corrected_be[1])
# print(sum(adata1.obs_names[1:3]==corrected_be[0].obs_names[1:3]))
# print(adata1.X.shape)
corrected_adata.var_names = corrected_be[0].var_names.tolist()
corrected_adata.obs_names = adata1.obs_names.tolist() + adata2.obs_names.tolist() 
corrected_adata.obs['cell_type'] = adata1.obs['cell_type'].tolist() + adata2.obs['cell_type'].tolist() 
corrected_adata.obs['batch'] = adata1.obs['batch'].tolist() + adata2.obs['batch'].tolist() 

# corrected_adata.obs_names = adata1.obs_names.tolist() + adata2.obs_names.tolist() 
# corrected_adata.obs['cell_type'] = adata1.obs['cell_type'].tolist() + adata2.obs['cell_type'].tolist() 
# corrected_adata.obs['batch'] = adata1.obs['batch'].tolist() + adata2.obs['batch'].tolist() 

corrected_adata

In [ ]:
# sum(adatads.obs['batch']=='batch2')

adata_ds = corrected_adata[adatads.obs_names,:].copy()
print(adata_ds)

npcs = 20  
sc.tl.pca(adata_ds, svd_solver='arpack', n_comps=npcs)  
adata_ds.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
adata_ds

In [ ]:
def save_output_pca(adata, save_dir): 
    # Save output of pca for evaluation ASW
    colnpc = []
    for i in range(20):
        colnpc.append("X_pca"+str(i+1))

    df = pd.DataFrame(adata.obsm['X_pca'][:, :20], columns=colnpc, index=adata.obs_names)
    df['batch'] = pd.Series(adata.obs['batch'], index=adata.obs_names)
#     df['batchlb'] = pd.Series(adata.obs['batchlb'], index=adata.obs_names)
    df['celltype'] = pd.Series(adata.obs['cell_type'], index=adata.obs_names)
    df.to_csv(save_dir+'scanorama_pca.csv')
    
save_output_pca(adata_ds, save_dir)
print('Save output of normalized pca in :',save_dir)    

In [ ]:
# def downsampling(cells_batches, percentds=0.1):
#     cells_ext = []
#     for i in range(len(cells_batches)):
#         rand_cidx = np.random.choice(cells_batches[i],size=int(len(cells_batches[i])*percentds),replace=False)
#         print('Extract cells from batch{}:  {} '.format(i+1,len(rand_cidx)))
#         cells_ext.append(rand_cidx)
        
#     return cells_ext    
        

# cells_batches = []
# # cells_batches.append(corrected_be[0].obs_names)
# # cells_batches.append(corrected_be[1].obs_names)

# cells_batches.append(adata1.obs_names)
# cells_batches.append(adata2.obs_names)
    
# cells_ext = downsampling(cells_batches, 0.1)

# cells_ext = np.concatenate(cells_ext)
# adata_ds = corrected_adata[cells_ext,:].copy()
# adata_ds    

# npcs = 20  
# sc.tl.pca(adata_ds, svd_solver='arpack', n_comps=npcs)  
# adata_ds.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
# adata_ds

In [ ]:
def plotUMAP(adata, color_group, save_filename='umap', save_dir='', npcs=20, nb_neighbors=15, use_repx = False):
    
    # Run with all genes and entire expr matrix
    if use_repx:
        sc.pp.neighbors(adata, use_rep='X')
    else:    # Run umap using pcs vectors
        sc.pp.neighbors(adata,n_neighbors=nb_neighbors, n_pcs=npcs)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename, save_dir)

    
npcs = 20  # our pre-defined
perplex = 30  # our pre-defined
nb_neighbors = 15
color_group = ['batch','cell_type'] 
save_fn_umap = 'scanorama_umap_ds_10percent_kokds'
print(save_fig_dir)
plotUMAP(adata_ds, color_group, save_fn_umap, save_fig_dir, npcs, nb_neighbors, False)
print('Save output of UMAP figure in :',save_fig_dir)

In [ ]:
def getExecutionTime(t1, t2, save_dir, usecase_name,filename):
    time_taken = (t2 - t1)
    time_taken_mins = divmod(time_taken, 60)
    time_taken_hours, rest = divmod( time_taken, 3600)
    hours_mins, hours_secs = divmod( rest, 60)
    print('Took seconds: '+str(timedelta(seconds=round(time_taken))))
    print('Took minutes: '+str(time_taken_mins))
    print('Took hours_minutes_seconds: ',str(time_taken_hours),str(hours_mins),str(hours_secs))
    
    

    data = {'use_case':usecase_name, 'exetime_secs':str(round(time_taken)),
           'exetimehours': str(time_taken_hours),
           'exetimemins': str(hours_mins),
           'exetimesecs':str(round(hours_secs))} 
    df = pd.DataFrame(data, index =['exetime'])
    print(df)
    df.to_csv(save_dir+filename) 

# Evaluation runtime of main batch effect removal function
filename = 'scanorama_exetime.csv'
usecase_name = 'scanorama_exetime' 
getExecutionTime(t1, t2, save_dir, usecase_name, filename)  # t1: start time, t2: end time  
print(save_dir)

In [ ]:
def save_output_umap(adata, save_dir): 
    colnu = []
    for i in range(adata.obsm['X_umap'].shape[1]):
        colnu.append("UMAP"+str(i+1))
    df = pd.DataFrame(adata.obsm['X_umap'], columns=colnu, index=adata.obs_names)
    df['batch'] = pd.Series(adata.obs['batch'], index=adata.obs_names)
#     df['batchlb'] = pd.Series(adata.obs['batchlb'], index=adata.obs_names)
    df['cell_type'] = pd.Series(adata.obs['cell_type'], index=adata.obs_names)
    df.to_csv(save_dir+'scanorama_umap.csv')  

    
# Apply to our data    
save_output_umap(adata_ds, save_dir)
print('Save output of normalized data in :',save_dir)

In [ ]:
# # Function to plot TSNE
def plotTSNE(adata, color_group, save_filename='tsne', save_dir='', n_pcs=20, perplex=30, use_repx = False):
    
    # Run with all genes and entire matrix
    if use_repx:
        sc.tl.tsne(adata, perplexity=perplex, use_rep='X')
    else:    # Run tsne using pcs vectors
        sc.tl.tsne(adata, n_pcs=n_pcs, perplexity=perplex)
    sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
    save_images(save_filename, save_dir) 

    
save_fn_tsne = 'scanorama_tsne_10percent_kokds'
plotTSNE(adata_ds, color_group, save_fn_tsne, save_fig_dir, npcs, perplex, False)
print('Save output of t-SNE in :',save_fig_dir)

In [ ]:
def save_output_tsne(adata, save_dir):   
    colnt = []  # Save output of tsne for visualization
    for i in range(adata.obsm['X_tsne'].shape[1]):
        colnt.append("tSNE_"+str(i+1))

    df = pd.DataFrame(adata.obsm['X_tsne'], columns=colnt, index=adata.obs_names)
    df['batch'] = pd.Series(adata.obs['batch'], index=adata.obs_names)
#     df['batchlb'] = pd.Series(adata.obs['batchlb'], index=adata.obs_names)
    df['celltype'] = pd.Series(adata.obs['cell_type'], index=adata.obs_names)
    df.to_csv(save_dir+'scanorama_tsne.csv') 

    
save_output_tsne(adata_ds, save_dir)
print('Save output of normalized data in :',save_dir)    

In [ ]:
savefn = 'scanorama_normalized_d8.h5ad'
corrected_adata.write_h5ad(os.path.join(save_dir,savefn))
print('Save output of normalized data in :',save_dir)

# savefn = 'scanorama_normalized_adata8.h5ad'
# adata.write_h5ad(os.path.join(save_dir,savefn))
# print('Save output of normalized data in :',save_dir)


In [ ]:
!pip freeze | grep scanorama
